# DFT + GWBSE Energy Calculation Using CH4

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.md)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
To run a DFT-GWBSE calculation we will use the [xtp_tools](https://votca.github.io/xtp_tools.html) calculator. Run the following command to view the help message of `xtp_tools`:

In [ ]:
!xtp_tools --help

### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Running a calculation with the default options
To run a DFT-GWBSE calculation we just need to provide the path to the file in XYZ with the molecular coordinates. Check the [dftgwbse defaults](https://votca.github.io/dftgwbse.html) for further information.

In [ ]:
!xtp_tools -n methane -t 2 -e dftgwbse > dftgwbse.log

The previous command will run the DFT-GWBSE calculation using the aforementioned defaults and the results are store in the *Current Work Directory* in a file named `methane_summary.xml`.

## Running a calculation using your own input file
Let create a folder to store the input `options` for XTP and copy the defaults in there

In [ ]:
!mkdir -p OPTIONFILES
!cp $VOTCASHARE/xtp/xml/dftgwbse.xml OPTIONFILES

You should have a *XML* file with the DFTWGSE options that looks like

In [ ]:
!head -n 10 OPTIONFILES/dftgwbse.xml

We will use the folling python snippet to change the `basisset` and the `auxbasisset`,

In [ ]:
from xml_editor import edit_calculator
edit_calculator("dftgwbse", "basisset", "ubecppol")
edit_calculator("dftgwbse", "auxbasisset", "aux-ubecppol")

Now you can run the calculation using the `-o` command line argument, as follows

In [ ]:
!xtp_tools -n methane -t 2 -e dftgwbse -o OPTIONFILES/dftgwbse.xml > dftgwbse.log

### Note
Alternatively you could have create an empty `dftgwbse.xml` file and add the `basisset` and the `auxbasisset` by hand, as follows,

```xml
<options>
  
  <dftgwbse help="Compute electronic excitations using GW-BSE">
    <basisset>ubecppol</basisset>
    <auxbasisset>aux-ubecppol</auxbasisset>
  </dftgwbse>

</options>
```

XTP will automatically compare the default values with the user-provided and overwrites the defaults with the user input. Also, If I given property does not have a default value you can provide one using the XML file described above.

## Partial Charges
We can compute now the partial charges using the `CHELPG` method by default. For more information see the [partialcharges documentation](https://votca.github.io/partialcharges.html). Once again, we only need to provide the name of the system to compute, which in our case is `methane`.

In [ ]:
!xtp_tools -n methane -e partialcharges

## Spectrum Calculation
Finally, lets compute a convolution of the singlet spectrum using a gaussian function. For doing so, we will modify the default values for the [spectrum calculator](https://votca.github.io/spectrum.html) to compute the spectrum between 9 and 25 eV, using 1000 points in that energy range. We will first grap a copy of [spectrum calculator](https://votca.github.io/spectrum.html) and the modify its values as follows,

In [ ]:
!cp $VOTCASHARE/xtp/xml/spectrum.xml OPTIONFILES
edit_calculator("spectrum", "lower", "9")
edit_calculator("spectrum", "upper", "25")
edit_calculator("spectrum", "points", "1000")

Finally, we can invoke the calculator

In [ ]:
!xtp_tools -n methane -e spectrum -o OPTIONFILES/spectrum.xml

The results are stored in the `methane_spectrum.dat` file.

## (Optional) Plot the spectrum

We will use [matplotlib](https://matplotlib.org/), [seaborn](https://seaborn.pydata.org/) and [pandas](https://pandas.pydata.org/) libraries to plot the spectrum. You can install it using [pip](https://pip.pypa.io/en/stable/) like

In [ ]:
!pip install seaborn --user

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
columns = ["E(eV)", "epsGaussian","IM(eps)Gaussian",  "epsLorentz",  "Im(esp)Lorentz"] 
df = pd.read_table("methane_spectrum.dat", comment="#", sep='\s+',names=columns) 
sns.relplot(x="E(eV)", y="epsGaussian", ci=None, kind="line", data=df) 
plt.plot()